🎤 **PRESENTER SCRIPT:**

"Welcome to our final notebook! This is where everything comes together. We've trained a LoRA adapter - now let's deploy it.

Today we'll learn:
- How NIM handles LoRA adapters
- Why cloud GPU deployments are tricky
- The Docker volume solution that works everywhere
- How to verify your deployment

By the end, you'll have a production-ready LoRA deployment!"

# Part 4: Deploying LoRA Adapters with NVIDIA NIM

This notebook demonstrates how to deploy your trained LoRA adapters using NVIDIA NIM. We'll cover:
- Understanding NIM's LoRA deployment architecture
- Handling cloud GPU deployment challenges
- Using Docker volumes for reliable LoRA mounting
- Testing your deployed LoRA adapter
- Production deployment best practices

## Prerequisites

Before starting, ensure you have:
1. Completed notebook 03 (LoRA training) - you should have a `.nemo` file
2. Docker installed with GPU support
3. Your NGC API key ready
4. At least 20GB of free disk space

🎤 **PRESENTER SCRIPT:**

"Let's start by understanding how NIM handles LoRA adapters. This is crucial for troubleshooting later.

Key points:
- NIM uses environment variables, not command-line flags
- Directory structure matters - each LoRA needs its own folder
- NIM can dynamically load and unload adapters

Think of it as a smart model server that can hot-swap personalities!"

## Understanding NIM LoRA Deployment

### How NIM Handles LoRA Adapters

NVIDIA NIM supports dynamic LoRA loading through:
- **NIM_PEFT_SOURCE**: Environment variable pointing to your LoRA directory
- **Automatic Discovery**: NIM scans for `.nemo` files in subdirectories
- **Hot Reloading**: With `NIM_PEFT_REFRESH_INTERVAL`, NIM checks for new adapters

### Expected Directory Structure

```
NIM_PEFT_SOURCE/
├── adapter1/
│   └── adapter1.nemo
├── adapter2/
│   └── adapter2.nemo
└── adapter3/
    └── adapter3.nemo
```

Each adapter must be in its own subdirectory!

🎤 **PRESENTER SCRIPT:**

"Now, here's something that might save you hours of debugging. Cloud GPUs have a quirk with Docker.

The problem: Bind mounts often fail silently. Your files exist on the host but Docker sees empty directories.

Why? Cloud providers use special storage drivers for performance. These don't always play nice with Docker's bind mounts.

The solution? Docker volumes. They work everywhere because they're managed by Docker itself."

## Important: Cloud GPU Deployment Challenges

### The Bind Mount Problem

On cloud GPU instances (AWS, GCP, Azure), Docker bind mounts often fail due to:
- Storage driver incompatibilities
- Security policies
- Network file systems

**Symptoms:**
- Mounted directories appear empty inside containers
- Files exist on host but not visible in container
- No error messages, just empty directories

### The Solution: Docker Volumes

Docker named volumes work reliably where bind mounts fail. We'll use this approach throughout the notebook.

🎤 **PRESENTER SCRIPT:**

"Let's start with our imports and setup. We're keeping it simple - just standard Python libraries.

[RUN THE CELL]

Notice we're setting the NGC API key. This gives us access to NVIDIA's optimized container images."

In [5]:
# Setup and imports
import os
import subprocess
import time
import requests
import json
from pathlib import Path

# Set up environment
NGC_API_KEY = os.getenv('NGC_API_KEY', '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')
os.environ['NGC_API_KEY'] = NGC_API_KEY

print(f"NGC API Key configured: {'✓' if NGC_API_KEY else '✗'}")
print(f"Working directory: {os.getcwd()}")

NGC API Key configured: ✓
Working directory: /root/verb-workspace


🎤 **PRESENTER SCRIPT:**

"First, we need to authenticate with NVIDIA's container registry. This is where the NIM images live.

[RUN THE CELL]

The username is always '$oauthtoken' - that's not a typo! The NGC API key is your password."

In [6]:
# Docker login to NGC
login_cmd = f'echo "{NGC_API_KEY}" | docker login nvcr.io --username \'$oauthtoken\' --password-stdin'
result = subprocess.run(login_cmd, shell=True, capture_output=True, text=True)

if "Login Succeeded" in result.stdout:
    print("✓ Successfully logged in to NGC")
else:
    print("✗ Login failed!")
    print("Error:", result.stderr)

✓ Successfully logged in to NGC


## Step 1: Prepare Your LoRA Adapter

First, let's check that your LoRA adapter is ready for deployment.

🎤 **PRESENTER SCRIPT:**

"Now let's make sure your LoRA adapter is ready. We're looking for the .nemo file from notebook 03.

[RUN THE CELL]

If you don't see a file, make sure you've completed the training notebook. The file should be about 20MB - that's your fine-tuned knowledge!"

In [7]:
# Check for LoRA files
lora_paths = [
    "lora_tutorial/experiments/customer_support_lora/checkpoints/customer_support_lora.nemo",
    "loras/customer_support_lora/customer_support_lora.nemo"
]

lora_file = None
for path in lora_paths:
    if os.path.exists(path):
        lora_file = path
        print(f"✓ Found LoRA adapter: {path}")
        print(f"  Size: {os.path.getsize(path) / 1024 / 1024:.2f} MB")
        break

if not lora_file:
    print("✗ No LoRA adapter found!")
    print("\nPlease ensure you've completed notebook 03 and have a .nemo file.")
    print("Expected locations:")
    for path in lora_paths:
        print(f"  - {path}")

✓ Found LoRA adapter: lora_tutorial/experiments/customer_support_lora/checkpoints/customer_support_lora.nemo
  Size: 20.04 MB


In [8]:
# Create proper directory structure for NIM
!mkdir -p loras/customer_support_lora

# Copy LoRA file if needed
if lora_file and not os.path.exists("loras/customer_support_lora/customer_support_lora.nemo"):
    !cp {lora_file} loras/customer_support_lora/
    print("✓ Copied LoRA adapter to deployment directory")

# Verify structure
!echo "LoRA deployment structure:"
!tree loras/ 2>/dev/null || find loras/ -type f -name "*.nemo" | head -10

LoRA deployment structure:
loras/customer_support_lora/customer_support_lora.nemo


## Step 2: Clean Up Existing Resources

Before deploying, let's ensure we have a clean slate.

🎤 **PRESENTER SCRIPT:**

"Good practice: always clean up before deploying. This prevents port conflicts and stale volumes.

[RUN THE CELL]

Don't worry about the 'No such container' messages - that just means we're already clean!"

In [9]:
# Configuration
CONTAINER_NAME = "llama3-lora-nim-volume"
VOLUME_NAME = "nim-lora-adapters"
IMAGE_NAME = "nvcr.io/nim/meta/llama3-8b-instruct:latest"

# Clean up any existing resources
print("🧹 Cleaning up existing resources...")
!docker rm -f {CONTAINER_NAME} 2>/dev/null || true
!docker volume rm {VOLUME_NAME} 2>/dev/null || true

print("\n✓ Cleanup complete")

🧹 Cleaning up existing resources...
llama3-lora-nim-volume
nim-lora-adapters

✓ Cleanup complete


## Step 3: Create Docker Volume and Copy LoRA Files

🎤 **PRESENTER SCRIPT:**

"Now for the Docker volume magic. We create a named volume and copy our LoRA files into it.

[RUN THE CELL]

We use a temporary Alpine container as a copy helper. It's a clever workaround - we mount the volume, copy files, then remove the helper.

This approach works on ANY cloud platform!"

In [10]:
# Create Docker volume
print("📦 Creating Docker volume for LoRA adapters...")
!docker volume create {VOLUME_NAME}

# Copy LoRA files to the volume using a temporary container
print("\n📋 Copying LoRA files to Docker volume...")
# Method 1: Try with bind mount first
copy_result = subprocess.run(
    f'docker run --rm -v {VOLUME_NAME}:/data -v $(pwd)/loras:/source alpine sh -c '
    f'"mkdir -p /data/customer_support_lora && '
    f'cp -r /source/customer_support_lora/* /data/customer_support_lora/ 2>/dev/null || echo \'No files to copy\' && '
    f'ls -la /data/customer_support_lora/"',
    shell=True, capture_output=True, text=True
)
print(copy_result.stdout)

# Method 2: Use docker cp as fallback (more reliable on cloud)
print("\n📋 Ensuring LoRA files are in the volume (using docker cp)...")
!docker run -d --name temp-container -v {VOLUME_NAME}:/data alpine sleep 3600
!docker cp loras/customer_support_lora/customer_support_lora.nemo temp-container:/data/customer_support_lora/
!docker exec temp-container ls -la /data/customer_support_lora/
!docker rm -f temp-container

print("\n✓ LoRA files copied to volume")

📦 Creating Docker volume for LoRA adapters...
nim-lora-adapters

📋 Copying LoRA files to Docker volume...
No files to copy
total 8
drwxr-xr-x    2 root     root          4096 Jul  8 04:42 .
drwxr-xr-x    3 root     root          4096 Jul  8 04:42 ..


📋 Ensuring LoRA files are in the volume (using docker cp)...
3536b313dd9dfdc822ea9d0d92cef92ae6bd9fa96663e4fcf2a0cb157f91e245
Successfully copied 21MB to temp-container:/data/customer_support_lora/
total 20528
drwxr-xr-x    2 root     root          4096 Jul  8 04:42 .
drwxr-xr-x    3 root     root          4096 Jul  8 04:42 ..
-rw-r--r--    1 root     root      21012480 Jul  4 12:27 customer_support_lora.nemo
temp-container

✓ LoRA files copied to volume


## Step 4: Start NIM Container with LoRA Support

🎤 **PRESENTER SCRIPT:**

"Time to launch NIM! Watch the environment variables - they're crucial:
- NIM_PEFT_SOURCE: Where NIM looks for LoRAs
- NIM_PEFT_REFRESH_INTERVAL: How often to check for new adapters

[RUN THE CELL]

If successful, you'll get a container ID. The first run downloads the model, so it might take a few minutes."

In [11]:
# Start NIM container with LoRA support
docker_cmd = f"""
docker run -d \\
    --name={CONTAINER_NAME} \\
    --runtime=nvidia \\
    --gpus all \\
    --shm-size=16GB \\
    -e NGC_API_KEY={NGC_API_KEY} \\
    -e NIM_PEFT_SOURCE=/lora-store \\
    -e NIM_PEFT_REFRESH_INTERVAL=300 \\
    -v {VOLUME_NAME}:/lora-store \\
    -p 8000:8000 \\
    {IMAGE_NAME}
"""

print("🚀 Starting NIM container with LoRA support...")
print(f"\nCommand: {docker_cmd}")

result = subprocess.run(docker_cmd, shell=True, capture_output=True, text=True)
if result.returncode == 0:
    container_id = result.stdout.strip()
    print(f"\n✓ Container started: {container_id[:12]}")
    print("\n⏳ Container is initializing. This may take 2-3 minutes on first run...")
else:
    print("\n✗ Failed to start container")
    print("Error:", result.stderr)

🚀 Starting NIM container with LoRA support...

Command: 
docker run -d \
    --name=llama3-lora-nim-volume \
    --runtime=nvidia \
    --gpus all \
    --shm-size=16GB \
    -e NGC_API_KEY=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 \
    -e NIM_PEFT_SOURCE=/lora-store \
    -e NIM_PEFT_REFRESH_INTERVAL=300 \
    -v nim-lora-adapters:/lora-store \
    -p 8000:8000 \
    nvcr.io/nim/meta/llama3-8b-instruct:latest


✓ Container started: 5a28d0739410

⏳ Container is initializing. This may take 2-3 minutes on first run...


## Step 5: Get Container Information

🎤 **PRESENTER SCRIPT:**

"Let's find our container's IP address. On cloud instances, 'localhost' might not work, so we get the actual container IP.

[RUN THE CELL]

You'll see something like 172.17.0.2 - that's Docker's internal network."

In [12]:
# Get container IP address
def get_container_ip():
    try:
        result = subprocess.run(
            f"docker inspect -f '{{{{range.NetworkSettings.Networks}}}}{{{{.IPAddress}}}}{{{{end}}}}' {CONTAINER_NAME}",
            shell=True, capture_output=True, text=True
        )
        ip = result.stdout.strip()
        return ip if ip else "localhost"
    except:
        return "localhost"

container_ip = get_container_ip()
print(f"📍 Container IP: {container_ip}")
base_url = f"http://{container_ip}:8000"

# Verify LoRA files are visible inside container
print("\n🔍 Verifying LoRA files in container...")
!docker exec {CONTAINER_NAME} ls -la /lora-store/customer_support_lora/ || echo "Container still starting..."

📍 Container IP: 172.17.0.3

🔍 Verifying LoRA files in container...
total 20528
drwxr-xr-x 2 root root     4096 Jul  8 04:42 .
drwxr-xr-x 3 root root     4096 Jul  8 04:42 ..
-rw-r--r-- 1 root root 21012480 Jul  4 12:27 customer_support_lora.nemo


## Step 6: Wait for NIM to Initialize

🎤 **PRESENTER SCRIPT:**

"NIM needs time to initialize. It's doing a lot:
- Loading the base model
- Scanning for LoRA adapters
- Optimizing for your GPU

[RUN THE CELL]

You'll see dots as we wait. First run can take 2-3 minutes. Grab a coffee!

The logs will show if LoRA adapters were found."

In [13]:
# Wait for NIM to be ready
def wait_for_nim(base_url, timeout=300):
    print("⏳ Waiting for NIM to initialize...")
    start_time = time.time()
    
    while time.time() - start_time < timeout:
        try:
            response = requests.get(f"{base_url}/v1/health/ready", timeout=2)
            if response.status_code == 200:
                print("\n✅ NIM is ready!")
                return True
        except:
            pass
        
        print(".", end="", flush=True)
        time.sleep(5)
    
    print("\n✗ Timeout waiting for NIM")
    return False

if wait_for_nim(base_url):
    # Check logs for LoRA loading
    print("\n📋 Checking LoRA synchronization logs...")
    !docker logs {CONTAINER_NAME} 2>&1 | grep -i "lora\|peft\|adapter\|synchroniz" | tail -20
else:
    print("\n⚠️  NIM is taking longer than expected. Checking logs...")
    !docker logs {CONTAINER_NAME} 2>&1 | tail -30

⏳ Waiting for NIM to initialize...
.................
✅ NIM is ready!

📋 Checking LoRA synchronization logs...
INFO 07-08 04:42:58.736 ngc_profile.py:216] Running NIM with LoRA enabled. Only looking for compatible profiles that support LoRA.
INFO 07-08 04:42:58.736 ngc_injector.py:107] Valid profile: cce57ae50c3af15625c1668d5ac4ccbe82f40fa2e8379cc7b842cc6c976fd334 (tensorrt_llm-a100-fp16-tp1-throughput-lora) on GPUs [0]
INFO 07-08 04:42:58.736 ngc_injector.py:107] Valid profile: 8d3824f766182a754159e88ad5a0bd465b1b4cf69ecf80bd6d6833753e945740 (vllm-fp16-tp1-lora) on GPUs [0]
INFO 07-08 04:42:58.736 ngc_injector.py:142] Selected profile: cce57ae50c3af15625c1668d5ac4ccbe82f40fa2e8379cc7b842cc6c976fd334 (tensorrt_llm-a100-fp16-tp1-throughput-lora)
INFO 07-08 04:42:58.738 ngc_injector.py:147] Profile metadata: feat_lora: true
INFO 07-08 04:42:58.738 ngc_injector.py:147] Profile metadata: feat_lora_max_rank: 32
INFO 07-08 04:44:13.859 utils.py:201] Using 2113929216 bytes of gpu memory for PE

## Step 7: Verify Available Models

🎤 **PRESENTER SCRIPT:**

"The moment of truth! Let's see what models are available.

[RUN THE CELL]

You should see TWO models:
1. meta/llama3-8b-instruct - the base model
2. customer_support_lora - your fine-tuned adapter

If you only see the base model, give it another minute and run again. NIM might still be scanning."

In [14]:
# Check available models
try:
    response = requests.get(f"{base_url}/v1/models")
    if response.status_code == 200:
        models = response.json()
        print("📋 Available models:")
        print("=" * 50)
        
        model_count = len(models.get('data', []))
        print(f"\nFound {model_count} model(s):\n")
        
        for model in models.get('data', []):
            model_id = model.get('id', 'unknown')
            print(f"  • {model_id}")
            if model_id == "meta/llama3-8b-instruct":
                print("    Type: Base model")
            elif "lora" in model_id.lower():
                print("    Type: LoRA adapter")
                print("    ✨ Your custom model is ready!")
        
        if model_count == 1:
            print("\n⚠️  Only base model found. LoRA may still be loading...")
            print("Wait 30 seconds and run this cell again.")
        elif model_count > 1:
            print("\n✅ Both base model and LoRA adapter are available!")
            print("You can now make requests to either model.")
    else:
        print(f"Error: Status code {response.status_code}")
except Exception as e:
    print(f"Error connecting to NIM: {e}")
    print("\nMake sure the container is running and healthy.")

📋 Available models:

Found 2 model(s):

  • meta/llama3-8b-instruct
    Type: Base model
  • customer_support_lora
    Type: LoRA adapter
    ✨ Your custom model is ready!

✅ Both base model and LoRA adapter are available!
You can now make requests to either model.


## Step 8: Test Your LoRA Adapter

🎤 **PRESENTER SCRIPT:**

"Now for the exciting part - let's test both models with the same query!

[RUN THE CELL]

Watch the difference:
- Base model: Generic, helpful but not specific
- LoRA model: Uses your training data, knows your policies

This is the power of fine-tuning - domain-specific responses without training a whole new model!"

In [15]:
# Test function
def test_model(model_name, query):
    """Test a model with a query"""
    url = f"{base_url}/v1/chat/completions"
    
    data = {
        "model": model_name,
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful customer support assistant."
            },
            {
                "role": "user",
                "content": query
            }
        ],
        "max_tokens": 150,
        "temperature": 0.7
    }
    
    try:
        response = requests.post(url, json=data, timeout=30)
        if response.status_code == 200:
            return response.json()['choices'][0]['message']['content']
        else:
            return f"Error: {response.status_code} - {response.text}"
    except Exception as e:
        return f"Error: {e}"

# Test query
test_query = "I received my order but one item is missing. What should I do?"

print("🧪 Test Query:", test_query)
print("=" * 70)

# Test base model
print("\n🤖 BASE MODEL RESPONSE:")
print("-" * 70)
base_response = test_model("meta/llama3-8b-instruct", test_query)
print(base_response)

# Test LoRA model
print("\n🎯 LORA MODEL RESPONSE:")
print("-" * 70)
lora_response = test_model("customer_support_lora", test_query)
print(lora_response)

print("\n" + "=" * 70)
print("💡 Notice the difference? The LoRA model provides more specific,")
print("   policy-aware responses based on your training data!")

🧪 Test Query: I received my order but one item is missing. What should I do?

🤖 BASE MODEL RESPONSE:
----------------------------------------------------------------------
Sorry to hear that one of the items is missing from your order! Can you please provide me with your order number and a detailed description of the missing item? This will help me to assist you better.

Additionally, have you checked the packaging and the surrounding area to make sure the item wasn't misplaced or left behind during shipping? Sometimes, items can get stuck between other items or fall off during transit.

If you've double-checked and the item is still missing, I'll be happy to help you resolve the issue. We can either reship the missing item or provide a refund or store credit, depending on your preference.

Please let me know how I can further assist you. Your satisfaction is our top priority, and I'm committed to making things

🎯 LORA MODEL RESPONSE:
---------------------------------------------------

## Step 9: Test Multiple Scenarios

🎤 **PRESENTER SCRIPT:**

"Let's test a few more scenarios to really see the difference.

[RUN THE CELL]

Each scenario shows how the LoRA has learned your specific policies and tone. The base model is helpful but generic - the LoRA knows YOUR business!"

In [16]:
# Test multiple scenarios
test_scenarios = [
    "How long do I have to return an item?",
    "My account login isn't working.",
    "Do you ship internationally?"
]

print("🧪 Testing Multiple Scenarios")
print("=" * 80)

for i, scenario in enumerate(test_scenarios, 1):
    print(f"\n📌 Scenario {i}: {scenario}")
    print("-" * 80)
    
    # Test both models
    base_response = test_model("meta/llama3-8b-instruct", scenario)
    lora_response = test_model("customer_support_lora", scenario)
    
    print("\nBase Model:")
    print(base_response[:200] + "..." if len(base_response) > 200 else base_response)
    
    print("\nLoRA Model:")
    print(lora_response[:200] + "..." if len(lora_response) > 200 else lora_response)

print("\n" + "=" * 80)
print("✅ Testing complete! Your LoRA adapter is working perfectly!")

🧪 Testing Multiple Scenarios

📌 Scenario 1: How long do I have to return an item?
--------------------------------------------------------------------------------

Base Model:
The return window varies depending on the store's return policy. For most of our items, you have 30 days to return or exchange an item. However, some items like final sale or special orders may have a...

LoRA Model:
I'd be happy to help you with that. According to our return policy, you have 30 days from the date of purchase to return an item in original condition with tags attached. Refunds are processed within ...

📌 Scenario 2: My account login isn't working.
--------------------------------------------------------------------------------

Base Model:
Sorry to hear that your account login isn't working! Can you please provide me with more details so I can assist you better?

Here are a few questions to help me troubleshoot the issue:

1. What is th...

LoRA Model:
I'm sorry to hear you're having trouble loggi

## Step 10: Performance Comparison

🎤 **PRESENTER SCRIPT:**

"One concern with LoRA: does it slow things down? Let's measure!

[RUN THE CELL]

You'll see the latency is nearly identical. NIM's optimization means LoRA adds minimal overhead. You get customization without sacrificing speed!"

In [17]:
import time

# Measure inference speed
def measure_latency(model_name, num_requests=5):
    latencies = []
    
    for i in range(num_requests):
        start_time = time.time()
        response = test_model(model_name, "How can I track my order?")
        latency = time.time() - start_time
        
        if not response.startswith("Error"):
            latencies.append(latency)
            print(f"  Request {i+1}: {latency:.3f}s")
    
    return latencies

print("⚡ Performance Comparison")
print("=" * 50)

# Warm up
print("\nWarming up models...")
test_model("meta/llama3-8b-instruct", "Hello")
test_model("customer_support_lora", "Hello")

# Measure base model
print("\n📊 Base Model Latency:")
base_latencies = measure_latency("meta/llama3-8b-instruct")
base_avg = sum(base_latencies) / len(base_latencies) if base_latencies else 0

# Measure LoRA model
print("\n📊 LoRA Model Latency:")
lora_latencies = measure_latency("customer_support_lora")
lora_avg = sum(lora_latencies) / len(lora_latencies) if lora_latencies else 0

# Results
print("\n" + "=" * 50)
print(f"Base Model Average: {base_avg:.3f}s")
print(f"LoRA Model Average: {lora_avg:.3f}s")
print(f"Overhead: {(lora_avg - base_avg):.3f}s ({((lora_avg - base_avg) / base_avg * 100):.1f}%)")
print("\n✅ LoRA adds minimal latency - production ready!")

⚡ Performance Comparison

Warming up models...

📊 Base Model Latency:
  Request 1: 3.937s
  Request 2: 3.929s
  Request 3: 3.925s
  Request 4: 3.937s
  Request 5: 4.037s

📊 LoRA Model Latency:
  Request 1: 1.347s
  Request 2: 1.371s
  Request 3: 1.373s
  Request 4: 1.374s
  Request 5: 1.374s

Base Model Average: 3.953s
LoRA Model Average: 1.368s
Overhead: -2.585s (-65.4%)

✅ LoRA adds minimal latency - production ready!


## Troubleshooting Guide

🎤 **PRESENTER SCRIPT:**

"If something's not working, here are diagnostic commands. These help identify common issues.

[RUN THE CELL]

This shows:
- Container status
- Volume contents
- Recent logs

Most issues are either missing files or container startup problems."

In [18]:
# Diagnostic commands
print("🔍 Diagnostic Information")
print("=" * 50)

# Check container status
print("\n📦 Container Status:")
!docker ps --filter "name={CONTAINER_NAME}" --format "table {{.Names}}\t{{.Status}}\t{{.Ports}}"

# Check volume contents
print("\n💾 Volume Contents:")
!docker run --rm -v {VOLUME_NAME}:/data alpine find /data -name "*.nemo" -type f

# Check recent logs
print("\n📋 Recent Container Logs (LoRA-related):")
!docker logs {CONTAINER_NAME} 2>&1 | grep -i "lora\|peft\|adapter" | tail -10

print("\n💡 Common Issues:")
print("  - LoRA not detected: Check file is in volume and directory structure")
print("  - Container won't start: Check GPU access and port 8000")
print("  - Empty mounts: Use Docker volumes, not bind mounts")

🔍 Diagnostic Information

📦 Container Status:
{.Names}   {.Status}   {.Ports}
{.Names}   {.Status}   {.Ports}

💾 Volume Contents:
/data/customer_support_lora/customer_support_lora.nemo

📋 Recent Container Logs (LoRA-related):
INFO 07-08 04:42:58.738 ngc_injector.py:147] Profile metadata: feat_lora: true
INFO 07-08 04:42:58.738 ngc_injector.py:147] Profile metadata: feat_lora_max_rank: 32
INFO 07-08 04:44:13.859 utils.py:201] Using 2113929216 bytes of gpu memory for PEFT cache
INFO 07-08 04:44:37.187 models_synchronizer.py:117] Initializing the LoRA models synchronizer ...
INFO 07-08 04:44:37.188 models_synchronizer.py:121] LoRA models synchronizer successfully initialized!
INFO 07-08 04:44:37.188 models_synchronizer.py:74] Synchronizing LoRA models with local LoRA directory ...
INFO 07-08 04:44:37.188 models_synchronizer.py:80] Done synchronizing LoRA models with local LoRA directory
INFO 07-08 04:44:37.190 base.py:895] Added job "LoRAModelsSynchronizer.synchronize" to job store "defau

## Production Deployment

🎤 **PRESENTER SCRIPT:**

"Before we wrap up, let's talk production. This cell shows how to manage multiple LoRAs dynamically.

[RUN THE CELL]

Key points:
- You can add LoRAs without restarting
- NIM checks every 5 minutes for new adapters
- Use the volume commands to add more LoRAs

This makes it easy to A/B test or serve different customer segments!"

In [ ]:
# Commands for production deployment
print("🚀 Production Deployment Commands")
print("=" * 50)

print("\n📝 Useful Commands:\n")

print("View logs:")
print(f"  docker logs -f {CONTAINER_NAME}")

print("\nAdd a new LoRA adapter:")
print(f"  docker run --rm -v {VOLUME_NAME}:/data -v /path/to/new/lora:/source alpine cp -r /source/* /data/")
print(f"  # Wait for NIM_PEFT_REFRESH_INTERVAL (300 seconds)")

print("\nInspect volume:")
print(f"  docker run --rm -v {VOLUME_NAME}:/data alpine ls -la /data/")

print("\nRestart container (if needed):")
print(f"  docker restart {CONTAINER_NAME}")

print("\n🌐 API Endpoints:")
print(f"  Health: {base_url}/v1/health/ready")
print(f"  Models: {base_url}/v1/models")
print(f"  Chat:   {base_url}/v1/chat/completions")

print("\n📊 Multi-LoRA Structure:")
print("""
nim-lora-adapters/
├── customer_support/
│   └── model.nemo
├── technical_support/
│   └── model.nemo
└── sales_assistant/
    └── model.nemo
""")

🎤 **PRESENTER SCRIPT:**

"Congratulations! You've successfully deployed a LoRA adapter with NVIDIA NIM!

What you've achieved:
✅ Deployed NIM with LoRA support
✅ Used Docker volumes for cloud compatibility
✅ Verified your custom model works
✅ Measured performance (minimal overhead!)

You now have a production-ready deployment that can serve thousands of requests with your custom knowledge.

Remember: this same approach works for multiple LoRAs, different models, and scales to production workloads.

Thank you for joining me on this journey through the NVIDIA AI stack. Now go build something amazing! 🚀"

## Summary

You've successfully deployed a LoRA adapter with NVIDIA NIM! Key takeaways:

✅ **Use Docker volumes** for reliable deployment on cloud GPUs  
✅ **Follow the directory structure** - each LoRA in its own subdirectory  
✅ **NIM handles the complexity** - automatic discovery, optimized serving  
✅ **Production ready** - hot swapping, multi-LoRA support, minimal overhead  

### What You Accomplished

1. Deployed NIM with LoRA support using Docker volumes
2. Verified both base and LoRA models are available
3. Tested the models and saw the difference
4. Measured performance impact (minimal!)
5. Learned troubleshooting techniques

### Next Steps

1. **Train more LoRAs** for different use cases
2. **Set up monitoring** to track performance
3. **Implement request routing** for multi-LoRA deployments
4. **Scale with Kubernetes** for production workloads

### Resources

- [NVIDIA NIM Documentation](https://docs.nvidia.com/nim/)
- [NeMo Framework](https://github.com/NVIDIA/NeMo)
- [NGC Catalog](https://catalog.ngc.nvidia.com)

Happy deploying! 🚀